In [64]:
#requirements

#install packages
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd


#load data
df=pd.read_csv('~/Desktop/train.csv') #2010筆 (train:1340 test:670)
#df=pd.read_csv('~/Desktop/bias_paper_train_mle.csv')  #5615筆 train:3715 test:1900
#df=pd.read_csv('~/Desktop/bias_paper_train_delete.csv')  #5613筆 train:3714 test:1899
#df=pd.read_csv('~/Desktop/outgroup_paper_train_mle.csv') #5307筆 train:3537 test:1770
#df=pd.read_csv('~/Desktop/outgroup_paper_train_delete.csv') #5305筆 train:3535 test:1770


data=df.values

np.random.seed(25)

#將資料打亂
np.random.shuffle(data)

'''
#bias_paper_train.csv
X=data[:,[2,3,4,5,6,7]]
y=data[:,1]
'''


#53 selfinvestment
#X=data[:,[49,53,54,56,58,61,62,63,64]]
X=data[:,[49,50,51,52,54,56,58,61,62,63,64]]
y=data[:,46]
#y=data[:,47]

train_num=1340
test_num=670

'''
X_train=X
X_test=X
y_train=y
y_test=y
'''

X_train=X[:train_num, :]
X_test=X[train_num:,:]
y_train=y[:train_num]
y_test=y[train_num:]


#X_train裡的元素是object 無法轉成tensor所以將其先轉為float
X_train=X_train.astype(float)
X_test=X_test.astype(float)
y_train=y_train.astype(float)
y_test=y_test.astype(float)

X_train=torch.Tensor(X_train)
X_test=torch.Tensor(X_test)
y_train=torch.Tensor(y_train)
y_test=torch.Tensor(y_test)

#建立資料與標籤一組的dataset
train_data=TensorDataset(X_train, y_train)
test_data=TensorDataset(X_test, y_test)

#指定資料的批次大小與建立Dataloader
size=10
load_train=DataLoader(train_data, batch_size=size, shuffle=True)
load_test=DataLoader(test_data, batch_size=size, shuffle=False) #test不用打亂資料

In [65]:
# 設置隨機種子
torch.manual_seed(32)

var_num=len(X[0])

#自訂的模型
class Quadratic(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        self.b=torch.nn.Parameter(torch.randn(()))
        self.W=nn.Parameter(torch.randn(1,var_num))
        self.Q=torch.nn.Parameter(torch.randn((var_num,var_num)))
        self.c=torch.nn.Parameter(torch.randn(()))
        self.d=torch.nn.Parameter(torch.randn(()))
        self.e=torch.nn.Parameter(torch.randn(()))


    def forward(self, X):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        return self.b+torch.matmul(self.W,X.T)+torch.diag(torch.matmul(torch.matmul(X,self.Q),X.T))

        '''
        return self.b+torch.matmul(self.W,X.T)+torch.diag(torch.matmul(torch.matmul(X,self.Q),X.T))\
+self.c*torch.log((4+torch.matmul(mat_1,X.T)))+self.d*torch.log((4+torch.matmul(mat_2,X.T)))\
+self.e*torch.exp(torch.matmul(mat_1,X.T))
        '''

    '''
    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'
    '''

In [66]:
#建立一個二次模型
model=Quadratic()

from torch import optim
from sklearn.metrics import r2_score
#建立loss function
#loss_fn=nn.SmoothL1Loss()
loss_fn=nn.L1Loss() #輸出需要long 所以我們在上面的tag加上 .type(torch.LongTensor)
#loss_fn=nn.MSELoss()
loss_fn=nn.HuberLoss()


'''
#Definition of LogCoshLoss
class LogCoshLoss(torch.nn.Module):

    def __init__(self):

        super(LogCoshLoss, self).__init__()



    def forward(self, x, target):

        logcosh = x + torch.nn.functional.softplus(-2 * x) - np.log(2)

        return torch.mean(logcosh)

loss_fn=LogCoshLoss()
'''

#建立優化函數
optimizer=optim.Adam(model.parameters(), lr=0.003,weight_decay=0.1)
#optimizer=optim.SGD(model.parameters(), lr=0.0002,weight_decay=0.1)

#開始訓練

def train(epoch):
    model.train() #將模型轉成訓練模式

    for data, targets in load_train:
        optimizer.zero_grad()
        outputs=model(data)
        outputs=outputs.reshape(size,)
        loss=loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()

    print ("epoch {}: 結束\n".format(epoch))
    
    model.eval()
    correct=0
    '''
    with torch.no_grad(): #不需計算微分
        for data, targets in load_train:
            outputs=model(data)
            outputs=outputs.reshape(1,)
            for i in range(len(outputs)):
                if abs((outputs[i]-targets[i]).item())<0.05:
                    correct+=1
        data_num=len(load_train.dataset)
        print("\n正確率:{}/{} ({:.0f}%)\n".format(correct,data_num,100.*correct/data_num))
    '''
    # 預測值
    y_train_pred = model(X_train)

    # 將預測值轉換為numpy陣列
    y_train_pred = y_train_pred.detach().numpy()
    y_train_pred=y_train_pred.reshape(train_num,)

    # 將實際目標值轉換為numpy陣列
    y_true = y_train.numpy()
    
    # 計算R^2 score
    r2 = r2_score(y_true, y_train_pred)
    print("Train R^2 score:", r2)

    # 預測值
    y_test_pred = model(X_test)

    # 將預測值轉換為numpy陣列
    y_test_pred = y_test_pred.detach().numpy()
    y_test_pred=y_test_pred.reshape(test_num,)

    # 將實際目標值轉換為numpy陣列
    y_true = y_test.numpy()

    # 計算R^2 score
    r2 = r2_score(y_true, y_test_pred)
    print("Test R^2 score:", r2)

'''
def test():
    model.eval()
    correct=0

    with torch.no_grad(): #不需計算微分
        for data, targets in load_test:
            #data is Tensor
            outputs=model(data)
            outputs=outputs.reshape(size,)
            for i in range(len(outputs)):
                if abs((outputs[i]-targets[i]).item())<1:
                    correct+=1
    data_num=len(load_test.dataset)
    print("\n正確率:{}/{} ({:.0f}%)\n".format(correct,data_num,100.*correct/data_num))
'''

'\ndef test():\n    model.eval()\n    correct=0\n\n    with torch.no_grad(): #不需計算微分\n        for data, targets in load_test:\n            #data is Tensor\n            outputs=model(data)\n            outputs=outputs.reshape(size,)\n            for i in range(len(outputs)):\n                if abs((outputs[i]-targets[i]).item())<1:\n                    correct+=1\n    data_num=len(load_test.dataset)\n    print("\n正確率:{}/{} ({:.0f}%)\n".format(correct,data_num,100.*correct/data_num))\n'

In [67]:


for epoch in range(500):
    train(epoch)


epoch 0: 結束

Train R^2 score: -35.557623663292574
Test R^2 score: -42.59779588250748
epoch 1: 結束

Train R^2 score: -19.10257711229545
Test R^2 score: -23.021115492890438
epoch 2: 結束

Train R^2 score: -10.268307085717913
Test R^2 score: -12.509825471109563
epoch 3: 結束

Train R^2 score: -5.352264262709595
Test R^2 score: -6.710270470937248
epoch 4: 結束

Train R^2 score: -2.555825265707555
Test R^2 score: -3.5645560980854123
epoch 5: 結束

Train R^2 score: -1.1032877962670122
Test R^2 score: -1.7432651786576159
epoch 6: 結束

Train R^2 score: -0.3720625681300842
Test R^2 score: -0.7618933097048113
epoch 7: 結束

Train R^2 score: 0.011289463972473435
Test R^2 score: -0.21943365472825782
epoch 8: 結束

Train R^2 score: 0.2768652481909021
Test R^2 score: 0.13175985043559746
epoch 9: 結束

Train R^2 score: 0.4203266491393215
Test R^2 score: 0.31747407407860284
epoch 10: 結束

Train R^2 score: 0.5005010351946761
Test R^2 score: 0.4356500450606037
epoch 11: 結束

Train R^2 score: 0.5592341822735072
Test R^2 s

In [68]:
# 預測值
y_train_pred = model(X_train)

# 將預測值轉換為numpy陣列
y_train_pred = y_train_pred.detach().numpy()
y_train_pred=y_train_pred.reshape(train_num,)
# 將實際目標值轉換為numpy陣列
y_true = y_train.numpy()

# 計算R^2 score
r2 = r2_score(y_true, y_train_pred)
print("Train R^2 score:", r2)

# 預測值
y_test_pred = model(X_test)

# 將預測值轉換為numpy陣列
y_test_pred = y_test_pred.detach().numpy()
y_test_pred=y_test_pred.reshape(test_num,)
# 將實際目標值轉換為numpy陣列
y_true = y_test.numpy()

# 計算R^2 score
r2 = r2_score(y_true, y_test_pred)
print("Test R^2 score:", r2)

#adjusted R^2


Train R^2 score: 0.6240629629369955
Test R^2 score: 0.595892795392591


In [69]:
from sklearn.metrics import mean_squared_error

# 计算RMSE
y_pred_train = model(X_train).detach().numpy()
y_train_numpy=y_train.detach().numpy()
y_pred_train=y_pred_train.reshape(train_num,)
rmse = mean_squared_error(y_train_numpy, y_pred_train, squared=False)
print(f"RMSE: {rmse}")

# 计算RMSE
y_pred_test= model(X_test).detach().numpy()
y_test_numpy=y_test.detach().numpy()
y_pred_test=y_pred_test.reshape(test_num,)

rmse = mean_squared_error(y_test_numpy, y_pred_test, squared=False)
print(f"RMSE: {rmse}")

RMSE: 0.8454186320304871
RMSE: 0.8738099932670593


In [70]:
print(model.b)
print(model.W)
print(model.Q)

Parameter containing:
tensor(-0.0690, requires_grad=True)
Parameter containing:
tensor([[ 0.2037,  0.3058,  0.1212,  0.0130, -0.2770, -0.1357,  0.0243,  0.2091,
          0.0211, -0.0511, -0.0752]], requires_grad=True)
Parameter containing:
tensor([[ 0.0024,  0.0217,  0.0246, -0.0005, -0.0047, -0.0098,  0.0331,  0.0010,
         -0.0084, -0.0469,  0.0079],
        [ 0.0217,  0.0341, -0.0194,  0.0159, -0.0253, -0.0037, -0.0049,  0.0088,
          0.0107,  0.0145,  0.0435],
        [ 0.0246, -0.0194,  0.0295, -0.0238,  0.0048,  0.0189,  0.0035, -0.0096,
         -0.0208, -0.0152, -0.0375],
        [-0.0005,  0.0159, -0.0238,  0.0183,  0.0163, -0.0238, -0.0262, -0.0092,
          0.0004, -0.0019, -0.0024],
        [-0.0047, -0.0253,  0.0048,  0.0163,  0.0629,  0.0105,  0.0139,  0.0082,
         -0.0035, -0.0189, -0.0218],
        [-0.0098, -0.0037,  0.0189, -0.0238,  0.0105, -0.0044, -0.0109, -0.0200,
          0.0206,  0.0048, -0.0256],
        [ 0.0331, -0.0049,  0.0035, -0.0262,  0.013